In [2]:
pip install scikeras

In [3]:
!pip install scikit-learn

In [11]:
# ✅ Import
import numpy as np
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models

# ✅ Step 1: Load CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(X_train.shape)

# ✅ Step 2: Preprocessing
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# ✅ Step 3: Define LightCNN Model
def create_lightcnn_model():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ✅ Step 4: Manual Bagging
n_estimators = 5
models_list = []

for i in range(n_estimators):
    print(f"\nTraining model {i+1}/{n_estimators}...")
    # Bootstrap sample
    idx = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_bootstrap = X_train[idx]
    y_bootstrap = y_train[idx]

    # Create and train model
    model = create_lightcnn_model()
    model.fit(X_bootstrap, y_bootstrap, epochs=20, batch_size=64, verbose=1)

    models_list.append(model)

# ✅ Step 5: Ensemble Prediction
# Collect predictions from each model
predictions = np.zeros((X_test.shape[0], 10))  # 10 classes

for model in models_list:
    preds = model.predict(X_test)
    predictions += preds

# Take average
final_preds = np.argmax(predictions, axis=1)

# ✅ Step 6: Evaluate
y_test = y_test.flatten()
accuracy = accuracy_score(y_test, final_preds)
print("\n✅ Final Bagging Ensemble Accuracy:", accuracy)


(40000, 32, 32, 3)

Training model 1/5...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.3124 - loss: 1.8685
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5255 - loss: 1.3351
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5940 - loss: 1.1442
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6448 - loss: 1.0262
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6764 - loss: 0.9277
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7046 - loss: 0.8574
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7266 - loss: 0.7959
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7498 - loss: 0.7335
Epoch 9/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7746 - loss: 0.6742
Epoch 10/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7851 - loss: 0.6420
Epoch 11/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7974 - loss: 0.6029
Epoch 12/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step

In [13]:
# Assuming you have your list of trained models:
# models_list = [model1, model2, ..., model5]

for i, model in enumerate(models_list, start=1):
    # Save the full model (architecture + weights + optimizer state)
    model.save(f"lightcnn_model_{i}.h5")
    print(f"Saved model #{i} to lightcnn_model_{i}.h5")


Saved model #1 to lightcnn_model_1.h5
Saved model #2 to lightcnn_model_2.h5
Saved model #3 to lightcnn_model_3.h5
Saved model #4 to lightcnn_model_4.h5
Saved model #5 to lightcnn_model_5.h5


In [14]:
import tensorflow as tf
print(tf.__version__)  # Should print without errors

2.18.0


In [15]:
import numpy as np
from tensorflow import keras

# Assuming your models are named lightcnn_model_1.h5, lightcnn_model_2.h5, etc.
loaded_models = []
for i in range(1, 6):  # Load 5 models
    model = keras.models.load_model(f"lightcnn_model_{i}.h5")
    loaded_models.append(model)
    print(f"Loaded model #{i} from lightcnn_model_{i}.h5")

# Collect predictions from each model
all_predictions = []

for i, model in enumerate(loaded_models, start=1):
    preds = model.predict(X_test, verbose=0)
    predicted_classes = np.argmax(preds, axis=1)
    all_predictions.append(predicted_classes)
    print(f"Model {i} predictions collected.")

# Convert list to 2D array: shape (5, num_samples)
all_predictions = np.array(all_predictions)

Loaded model #1 from lightcnn_model_1.h5
Loaded model #2 from lightcnn_model_2.h5
Loaded model #3 from lightcnn_model_3.h5
Loaded model #4 from lightcnn_model_4.h5


Loaded model #5 from lightcnn_model_5.h5
Model 1 predictions collected.
Model 2 predictions collected.
Model 3 predictions collected.
Model 4 predictions collected.
Model 5 predictions collected.


In [16]:
from scipy.stats import mode

# Transpose to shape (num_samples, 5) to vote across models for each sample
voted_predictions, _ = mode(all_predictions, axis=0)
voted_predictions = voted_predictions.flatten()


In [17]:
from sklearn.metrics import accuracy_score

# If y_test is shape (num_samples, 1), flatten it
y_test_flat = y_test.flatten()

ensemble_accuracy = accuracy_score(y_test_flat, voted_predictions)
print(f"🧠 Ensemble Voting Accuracy: {ensemble_accuracy * 100:.2f}%")


🧠 Ensemble Voting Accuracy: 70.00%
